### Root Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [5]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Root Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_pair = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [10]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [11]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]
#disable_list = []

In [12]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [13]:
word_list = list((word_select_set.union(words_set)).difference(disable_word_set))
len(word_list)

200

In [14]:
df_word_result = pd.DataFrame(word_list, columns=["word"])
df_word_result

,word
0,burada
1,bak
2,ben
3,olduğunu
4,biliyorum
...,...
195,git
196,önce
197,iyi
198,olarak


In [15]:
df_word_result = strip(df_word_result)
df_word_result

,word
0,burada
1,bak
2,ben
3,olduğunu
4,biliyorum
...,...
195,git
196,önce
197,iyi
198,olarak


In [16]:
df_lemma_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Data/{lang_folder.capitalize()}/Turkish_100000_Word_All_Lemma_Result.xlsx")
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [17]:
df_lemma_all = strip(df_lemma_all)
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [18]:
for column in df_lemma_all.columns:
    column_nunique = df_lemma_all[f"{column}"].nunique()
    print(f"{column} : {column_nunique}")

word : 100000
lemma.atis : 99386
lemma.boun : 86986
lemma.framenet : 96794
lemma.imst : 93539
lemma : 88593
lemma.kenet : 85433
lemma.penn : 88593
lemma.spacylookup : 28112
lemma.tourism : 100000


In [19]:
#df_lemma_all = df_lemma_all.iloc[word_start:word_end,]

In [20]:
df_lemma_select = df_lemma_all[["word","lemma.spacylookup"]]

In [21]:
df_lemma_select["lemma.spacylookup"].nunique()

28112

In [22]:
df_word_lemma_merge = pd.merge(df_word_result, df_lemma_select, how="left", on="word")
df_word_lemma_merge.drop_duplicates(inplace=True)
df_word_lemma_merge

,word,lemma.spacylookup
0,burada,bura
1,bak,bak
2,ben,ben
3,olduğunu,ol
4,biliyorum,bil
...,...,...
195,git,git
196,önce,önce
197,iyi,iyi
198,olarak,ol


In [23]:
df_word_lemma_join = df_word_lemma_merge.groupby(["lemma.spacylookup"])["word"].apply(", ".join).reset_index()
df_word_lemma_join

,lemma.spacylookup,word
0,adam,adam
1,al,al
2,ama,ama
3,anne,anne
4,artık,artık
...,...,...
142,üç,üç
143,şekil,şekilde
144,şey,"şey, şeyler, şeyi"
145,şimdi,şimdi


In [24]:
lemma_length = len(df_word_lemma_join)

In [25]:
df_word_lemma_join.to_excel(f"{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx", index=False)

#### Copy Move And Delete

In [26]:
output_file = glob.glob(f"{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx")
output_file

['200_Word_147_Lemma_And_Join_Words.xlsx']

In [27]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [28]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [28]:
import re
import pandas as pd
import numpy as np
import glob

In [29]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/4-Root Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

In [30]:
df_40000 = pd.read_excel(f"{path}/40000_Word_11278_Lemma_And_Join_Words.xlsx")
df_40000

,lemma.spacylookup,word
0,a,a
1,aa,aa
2,aaa,aaa
3,aaah,aaah
4,aah,aah
...,...,...
11273,şınav,şınav
11274,şırınga,şırınga
11275,şşt,şşt
11276,şşş,şşş


In [31]:
df_20000 = pd.read_excel(f"{path}/20000_Word_6020_Lemma_And_Join_Words.xlsx")
df_20000

,lemma.spacylookup,word
0,a,a
1,aa,aa
2,aaa,aaa
3,aaah,aaah
4,aah,aah
...,...,...
6015,şüphelimiz,şüphelimiz
6016,şüphesiz,şüphesiz
6017,şık,şık
6018,şımarık,şımarık


In [32]:
df_10000 = pd.read_excel(f"{path}/10000_Word_3298_Lemma_And_Join_Words.xlsx")
df_10000

,lemma.spacylookup,word
0,a,a
1,aa,aa
2,aaa,aaa
3,aah,aah
4,abd,abd
...,...,...
3293,şükür,"şükürler, şükür"
3294,şüphe,"şüphem, şüphe"
3295,şüpheli,"şüpheliyim, şüpheli"
3296,şüphesiz,şüphesiz


In [33]:
df_5000 = pd.read_excel(f"{path}/5000_Word_1922_Lemma_And_Join_Words.xlsx")
df_5000

,lemma.spacylookup,word
0,a,a
1,aa,aa
2,aah,aah
3,abd,abd
4,abi,abi
...,...,...
1917,şükür,"şükürler, şükür"
1918,şüphe,şüphe
1919,şüpheli,şüpheli
1920,şüphesiz,şüphesiz


In [34]:
df_1000 = pd.read_excel(f"{path}/1000_Word_547_Lemma_And_Join_Words.xlsx")
df_1000

,lemma.spacylookup,word
0,a,a
1,acaba,acaba
2,acele,acele
3,acil,acil
4,acı,acı
...,...,...
542,şey,"şeyler, şeyden, şeye, şeyleri, şey, şeyi, şeyin"
543,şimdi,şimdi
544,şimdiye,şimdiye
545,şu,"şunu, şu, şuna"


In [35]:
df_200 = pd.read_excel(f"{path}/200_Word_147_Lemma_And_Join_Words.xlsx")
df_200

,lemma.spacylookup,word
0,adam,adam
1,al,al
2,ama,ama
3,anne,anne
4,artık,artık
...,...,...
142,üç,üç
143,şekil,şekilde
144,şey,"şey, şeyler, şeyi"
145,şimdi,şimdi


In [36]:
df_lemma_word = df_40000.iloc[:,[0]]
df_lemma_word

,lemma.spacylookup
0,a
1,aa
2,aaa
3,aaah
4,aah
...,...
11273,şınav
11274,şırınga
11275,şşt
11276,şşş


In [37]:
df_merge_200 = pd.merge(df_lemma_word, df_200, how="left", on="lemma.spacylookup")
df_merge_200.drop_duplicates(inplace=True)
df_merge_200.rename(columns={"word":"word_200"}, inplace=True)
df_merge_200

,lemma.spacylookup,word_200
0,a,NaN
1,aa,NaN
2,aaa,NaN
3,aaah,NaN
4,aah,NaN
...,...,...
11273,şınav,NaN
11274,şırınga,NaN
11275,şşt,NaN
11276,şşş,NaN


In [38]:
df_merge_1000 = pd.merge(df_merge_200, df_1000, how="left", on="lemma.spacylookup")
df_merge_1000.drop_duplicates(inplace=True)
df_merge_1000.rename(columns={"word":"word_1000"}, inplace=True)
df_merge_1000

,lemma.spacylookup,word_200,word_1000
0,a,NaN,a
1,aa,NaN,NaN
2,aaa,NaN,NaN
3,aaah,NaN,NaN
4,aah,NaN,NaN
...,...,...,...
11273,şınav,NaN,NaN
11274,şırınga,NaN,NaN
11275,şşt,NaN,NaN
11276,şşş,NaN,NaN


In [39]:
df_merge_5000 = pd.merge(df_merge_1000, df_5000, how="left", on="lemma.spacylookup")
df_merge_5000.drop_duplicates(inplace=True)
df_merge_5000.rename(columns={"word":"word_5000"}, inplace=True)
df_merge_5000

,lemma.spacylookup,word_200,word_1000,word_5000
0,a,NaN,a,a
1,aa,NaN,NaN,aa
2,aaa,NaN,NaN,NaN
3,aaah,NaN,NaN,NaN
4,aah,NaN,NaN,aah
...,...,...,...,...
11273,şınav,NaN,NaN,NaN
11274,şırınga,NaN,NaN,NaN
11275,şşt,NaN,NaN,NaN
11276,şşş,NaN,NaN,NaN


In [40]:
df_merge_10000 = pd.merge(df_merge_5000, df_10000, how="left", on="lemma.spacylookup")
df_merge_10000.drop_duplicates(inplace=True)
df_merge_10000.rename(columns={"word":"word_10000"}, inplace=True)
df_merge_10000

,lemma.spacylookup,word_200,word_1000,word_5000,word_10000
0,a,NaN,a,a,a
1,aa,NaN,NaN,aa,aa
2,aaa,NaN,NaN,NaN,aaa
3,aaah,NaN,NaN,NaN,NaN
4,aah,NaN,NaN,aah,aah
...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN
11274,şırınga,NaN,NaN,NaN,NaN
11275,şşt,NaN,NaN,NaN,NaN
11276,şşş,NaN,NaN,NaN,NaN


In [41]:
df_merge_20000 = pd.merge(df_merge_10000, df_20000, how="left", on="lemma.spacylookup")
df_merge_20000.drop_duplicates(inplace=True)
df_merge_20000.rename(columns={"word":"word_20000"}, inplace=True)
df_merge_20000

,lemma.spacylookup,word_200,word_1000,word_5000,word_10000,word_20000
0,a,NaN,a,a,a,a
1,aa,NaN,NaN,aa,aa,aa
2,aaa,NaN,NaN,NaN,aaa,aaa
3,aaah,NaN,NaN,NaN,NaN,aaah
4,aah,NaN,NaN,aah,aah,aah
...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN
11274,şırınga,NaN,NaN,NaN,NaN,NaN
11275,şşt,NaN,NaN,NaN,NaN,NaN
11276,şşş,NaN,NaN,NaN,NaN,şşş


In [42]:
df_merge_40000 = pd.merge(df_merge_20000, df_40000, how="left", on="lemma.spacylookup")
df_merge_40000.drop_duplicates(inplace=True)
df_merge_40000.rename(columns={"word":"word_40000"}, inplace=True)
df_merge_40000

,lemma.spacylookup,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,a,a,a,a
1,aa,NaN,NaN,aa,aa,aa,aa
2,aaa,NaN,NaN,NaN,aaa,aaa,aaa
3,aaah,NaN,NaN,NaN,NaN,aaah,aaah
4,aah,NaN,NaN,aah,aah,aah,aah
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,şşş


In [43]:
df_merge_40000.rename(columns={"lemma.spacylookup":"lemma_word"}, inplace=True)
df_merge_40000

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,a,a,a,a
1,aa,NaN,NaN,aa,aa,aa,aa
2,aaa,NaN,NaN,NaN,aaa,aaa,aaa
3,aaah,NaN,NaN,NaN,NaN,aaah,aaah
4,aah,NaN,NaN,aah,aah,aah,aah
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,şşş


In [47]:
df_merge_40000.to_excel(f"{path}/Turkish_200_To_40000_Word_Lemma_Merge_Result.xlsx", index=False)

#### Lemma Merge Cumulative

In [44]:
df_200_40000 = pd.read_excel(f"{path}/Turkish_200_To_40000_Word_Lemma_Merge_Result.xlsx")
df_200_40000

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,a,a,a,a
1,aa,NaN,NaN,aa,aa,aa,aa
2,aaa,NaN,NaN,NaN,aaa,aaa,aaa
3,aaah,NaN,NaN,NaN,NaN,aaah,aaah
4,aah,NaN,NaN,aah,aah,aah,aah
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,şşş


In [45]:
def float_detect(words):
    #word_out = []
    try:
        word_out = [word.strip() for word in words.split(",")]
        #for word in words.split(","):
        #    word_var = word.strip()
        #    word_out.append(word_var)
    except:
        word_out = []
    return word_out

In [46]:
for i in range(len(df_200_40000)):
    word_200 = df_200_40000.loc[i,"word_200"]
    word_1000 = df_200_40000.loc[i,"word_1000"]
    word_5000 = df_200_40000.loc[i,"word_5000"]
    word_10000 = df_200_40000.loc[i,"word_10000"]
    word_20000 = df_200_40000.loc[i,"word_20000"]
    word_40000 = df_200_40000.loc[i,"word_40000"]
    set_200 = set(float_detect(word_200))
    set_1000 = set(float_detect(word_1000))
    set_5000 = set(float_detect(word_5000))
    set_10000 = set(float_detect(word_10000))
    set_20000 = set(float_detect(word_20000))
    set_40000 = set(float_detect(word_40000))
    df_200_40000.loc[i,"word_40000"] = ", ".join(list(set_40000.difference(set_20000)))
    df_200_40000.loc[i,"word_20000"] = ", ".join(list(set_20000.difference(set_10000)))
    df_200_40000.loc[i,"word_10000"] = ", ".join(list(set_10000.difference(set_5000)))
    df_200_40000.loc[i,"word_5000"] = ", ".join(list(set_5000.difference(set_1000)))
    df_200_40000.loc[i,"word_1000"] = ", ".join(list(set_1000.difference(set_200)))

In [47]:
df_200_40000["word_1000"] = df_200_40000["word_1000"].apply(lambda x: np.nan if x == "" else x)
df_200_40000["word_5000"] = df_200_40000["word_5000"].apply(lambda x: np.nan if x == "" else x)
df_200_40000["word_10000"] = df_200_40000["word_10000"].apply(lambda x: np.nan if x == "" else x)
df_200_40000["word_20000"] = df_200_40000["word_20000"].apply(lambda x: np.nan if x == "" else x)
df_200_40000["word_40000"] = df_200_40000["word_40000"].apply(lambda x: np.nan if x == "" else x)

In [48]:
df_200_40000

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,NaN,NaN,NaN,NaN
1,aa,NaN,NaN,aa,NaN,NaN,NaN
2,aaa,NaN,NaN,NaN,aaa,NaN,NaN
3,aaah,NaN,NaN,NaN,NaN,aaah,NaN
4,aah,NaN,NaN,aah,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,NaN


In [52]:
df_200_40000

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,NaN,NaN,NaN,NaN
1,aa,NaN,NaN,aa,NaN,NaN,NaN
2,aaa,NaN,NaN,NaN,aaa,NaN,NaN
3,aaah,NaN,NaN,NaN,NaN,aaah,NaN
4,aah,NaN,NaN,aah,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,NaN


In [53]:
df_200_40000.to_excel(f"{path}/Turkish_200_To_40000_Word_Lemma_Cumulative_Result.xlsx", index=False)

#### Adding Pos Tag

In [54]:
df_200_40000 = pd.read_excel(f"{path}/Turkish_200_To_40000_Word_Lemma_Cumulative_Result.xlsx")
df_200_40000

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,a,NaN,a,NaN,NaN,NaN,NaN
1,aa,NaN,NaN,aa,NaN,NaN,NaN
2,aaa,NaN,NaN,NaN,aaa,NaN,NaN
3,aaah,NaN,NaN,NaN,NaN,aaah,NaN
4,aah,NaN,NaN,aah,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,şşş,NaN,NaN,NaN,NaN,şşş,NaN


In [59]:
df_pos_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Data/{lang_folder.capitalize()}/Turkish_100000_Word_All_Pos_Result.xlsx")
df_pos_all = df_pos_all[["word","pos.ud_imst"]]
df_pos_all.rename(columns={"word":"lemma_word","pos.ud_imst":"pos_tag"}, inplace=True)
df_pos_all

,lemma_word,pos_tag
0,bir,NUM
1,bu,DET
2,ne,PRON
3,ve,CCONJ
4,için,ADP
...,...,...
99995,casusa,NOUN
99996,seppuku,NOUN
99997,tırmandığını,NOUN
99998,çözülemeyecek,VERB


In [60]:
df_200_40000_pos_merge = pd.merge(df_200_40000,df_pos_all,how="left",on="lemma_word")
df_200_40000_pos_merge.drop_duplicates(inplace=True)
df_200_40000_pos_merge

,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000,pos_tag
0,a,NaN,a,NaN,NaN,NaN,NaN,INTJ
1,aa,NaN,NaN,aa,NaN,NaN,NaN,NOUN
2,aaa,NaN,NaN,NaN,aaa,NaN,NaN,NOUN
3,aaah,NaN,NaN,NaN,NaN,aaah,NaN,NOUN
4,aah,NaN,NaN,aah,NaN,NaN,NaN,NOUN
...,...,...,...,...,...,...,...,...
11273,şınav,NaN,NaN,NaN,NaN,NaN,şınav,NOUN
11274,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga,NOUN
11275,şşt,NaN,NaN,NaN,NaN,NaN,şşt,NOUN
11276,şşş,NaN,NaN,NaN,NaN,şşş,NaN,NOUN


In [61]:
df_200_40000_pos_merge = df_200_40000_pos_merge[["pos_tag","lemma_word","word_200","word_1000","word_5000","word_10000","word_20000","word_40000",]]
df_200_40000_pos_merge

,pos_tag,lemma_word,word_200,word_1000,word_5000,word_10000,word_20000,word_40000
0,INTJ,a,NaN,a,NaN,NaN,NaN,NaN
1,NOUN,aa,NaN,NaN,aa,NaN,NaN,NaN
2,NOUN,aaa,NaN,NaN,NaN,aaa,NaN,NaN
3,NOUN,aaah,NaN,NaN,NaN,NaN,aaah,NaN
4,NOUN,aah,NaN,NaN,aah,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11273,NOUN,şınav,NaN,NaN,NaN,NaN,NaN,şınav
11274,NOUN,şırınga,NaN,NaN,NaN,NaN,NaN,şırınga
11275,NOUN,şşt,NaN,NaN,NaN,NaN,NaN,şşt
11276,NOUN,şşş,NaN,NaN,NaN,NaN,şşş,NaN


In [62]:
df_200_40000_pos_merge.to_excel(f"{path}/Turkish_200_To_40000_Pos_Word_Lemma_Cumulative_Result.xlsx", index=False)